# Parrot Prediction Courses

## Hyper-parameter tuning with grid search

As you know there are plenty of tunable parameters. Each one results in different output. The question is which combination results in best output.

The following notebook will show you how to configure Scikit-learn grid search module for figuring out the best parameters for your XGBoost model.

**What you will learn:**
- Finding best hyper-parameters for your dataset

Let's begin with loading all required libraries in one place and set seed number for reproducibility.

In [1]:
import numpy as np

from xgboost.sklearn import XGBClassifier

from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.cross_validation import StratifiedKFold

# reproducibility
seed = 123
np.random.seed(seed)

Generate artificial dataset

In [2]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=8, n_redundant=3, n_repeated=2, random_state=seed)

Define cross-validation strategy for testing. Let's use `StratifiedKFold` which guarantees that target label is equally distributed across each fold.

In [3]:
cv = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=seed)

Define a dictionary holding possible parameter values we want to test.

In [17]:
params_grid = {
    'max_depth': [1, 2, 3],
    'n_estimators': [5, 10, 25, 50],
    'learning_rate': np.linspace(1e-16, 1, 3)
}

And a dictiorany for fixed parameters.

In [12]:
params_fixed = {
    'objective': 'binary:logistic',
    'silent': 1
}

Create a `GridSearchCV` estimator. We will be looking for combination giving the best accuracy.

In [18]:
bst_grid = GridSearchCV(
    estimator=XGBClassifier(**params_fixed, seed=seed),
    param_grid=params_grid,
    cv=cv,
    scoring='accuracy'
)

Before running the calcuations notice that $3*4*3*10=360$ models will be created for testing all combinations. You should always have rough estimations about what is going to happen.

In [19]:
bst_grid.fit(X, y)

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 0 ..., 0 1], n_folds=10, shuffle=True, random_state=123),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=123, silent=1, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': array([  1.00000e-16,   5.00000e-01,   1.00000e+00]), 'max_depth': [2, 3, 4], 'n_estimators': [5, 10, 25, 50]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

Now, we can look at all obtained scores, and try to manually see what matters and what not. A quick glance looks that the largeer `n_estimators` then the accuracy is higher.

In [20]:
bst_grid.grid_scores_

[mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 2, 'n_estimators': 5},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 2, 'n_estimators': 10},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 2, 'n_estimators': 25},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 2, 'n_estimators': 50},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 3, 'n_estimators': 5},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 3, 'n_estimators': 10},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 3, 'n_estimators': 25},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.9999999999999998e-17, 'max_depth': 3, 'n_estimators': 50},
 mean: 0.50000, std: 0.00000, params: {'learning_rate': 9.99999999

If there are many results, we can sort or filter them manually or get best combination

In [21]:
print("Best accuracy obtained: {0}".format(bst_grid.best_score_))
print("Parameters:")
for key, value in bst_grid.best_params_.items():
    print("\t{}: {}".format(key, value))

Best accuracy obtained: 0.931
Parameters:
	learning_rate: 0.5
	max_depth: 3
	n_estimators: 50


Looking for best parameters is an iterative process. You should start with coarsed-granularity and move to to more detailed values.